In [26]:
import os, requests, torch, transformers
from PIL import Image
from torchvision import transforms
from io import BytesIO

os.environ['TRANSFORMERS_CACHE'] = '/scratch1/nic261/hf_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/scratch1/nic261/hf_cache'

ckpt_name = 'aehrc/mimic-cxr-report-gen-single'

encoder_decoder = transformers.AutoModel.from_pretrained(ckpt_name, trust_remote_code=True)
tokenizer = transformers.PreTrainedTokenizerFast.from_pretrained(ckpt_name)
image_processor = transformers.AutoFeatureExtractor.from_pretrained(ckpt_name)

test_transforms = transforms.Compose(
    [
        transforms.Resize(size=image_processor.size['shortest_edge']),
        transforms.CenterCrop(size=[
            image_processor.size['shortest_edge'],
            image_processor.size['shortest_edge'],
        ]
        ),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=image_processor.image_mean,
            std=image_processor.image_std,
        ),
    ]
)

/datasets/work/hb-mlaifsp-mm/work/environments/cxrmate/lib/python3.9/site-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [27]:
url = 'https://www.stritch.luc.edu/lumen/meded/radio/curriculum/IPM/PCM/86a_labelled.jpg'
response = requests.get(url)
image_a = Image.open(BytesIO(response.content))
image_a = image_a.convert('RGB')
image_a = test_transforms(image_a)

url = 'https://prod-images-static.radiopaedia.org/images/566180/d527ff6fc1482161c9225345c4ab42_big_gallery.jpg'
response = requests.get(url)
image_b = Image.open(BytesIO(response.content))
image_b = image_b.convert('RGB')
image_b = test_transforms(image_b)

images = torch.stack([image_a, image_b], dim=0)
images.shape


torch.Size([2, 3, 384, 384])

In [30]:
outputs = encoder_decoder.generate(
    pixel_values=images,
    special_token_ids=[tokenizer.sep_token_id],
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    return_dict_in_generate=True,
    use_cache=True,
    max_length=256,
    num_beams=4,
)

findings, impression = encoder_decoder.split_and_decode_sections(
    outputs.sequences,
    [tokenizer.sep_token_id, tokenizer.eos_token_id],
    tokenizer,
)

In [31]:
for i, j in zip(findings, impression):
    print(f'Findings: {i}\nImpression: {j}\n')

Findings: There is a moderate left pleural effusion, slightly increased since ___. There is adjacent compressive atelectasis. The right lung is clear. There is no pneumothorax. The heart and mediastinum are within normal limits.
Impression: Moderate left pleural effusion, slightly increased since ___.

Findings: The patient is status post median sternotomy and mitral valve replacement. The heart is moderately enlarged, and the patient is status post mitral valve replacement. The aorta is tortuous and diffusely calcified. There is mild pulmonary vascular congestion, improved compared to the prior study. Small right pleural effusion is unchanged. No pneumothorax is identified. There are no acute osseous abnormalities.
Impression: Mild pulmonary vascular congestion, improved compared to the prior study. Persistent small right pleural effusion.

